In [170]:
import spacy

In [173]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9MB 13.1MB/s eta 0:00:01    |█▊                              | 44.2MB 2.7MB/s eta 0:04:49     |█▊                              | 44.3MB 2.7MB/s eta 0:04:49     |█████████▍                      | 242.3MB 37.2MB/s eta 0:00:16     |████████████▌                   | 322.4MB 3.0MB/s eta 0:02:48     |████████████▋                   | 325.5MB 8.4MB/s eta 0:01:01     |█████████████████               | 442.3MB 2.0MB/s eta 0:03:14     |██████████████████████████▌     | 686.3MB 5.7MB/s eta 0:00:25
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp37-none-any.whl size=829180945 sha256=f074a7a73bf2b376231f70e310d901a9512e7ab848075185d454fea5c730430c
  Stored in directory: /private/var/folders/xy/93wzwgn579b9nnb78hvh7g740000gn/T/pip-ephem-wheel-cache-vuu911lc/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load(

In [174]:
nlp = spacy.load('en_core_web_lg')

In [175]:
quote_matcher = Matcher(nlp.vocab)
quote_matcher.add('QUOTED', None, [{'ORTH': "'"}, {'IS_ALPHA': True, 'OP': '+'}, {'ORTH': "'"}])
def quote_merger(doc):
    # this will be called on the Doc object in the pipeline
    matched_spans = []
    matches = quote_matcher(doc)
    for match_id, start, end in matches:
        span = doc[start:end]
        matched_spans.append(span)
    for span in matched_spans:  # merge into one token after collecting all matches
        span.merge()
    return doc

nlp.add_pipe(quote_merger, first=True)

In [187]:
TEXTS = [
#     "You are Justin, a knight living in the kingdom of Izir. You have a steel longsword and a wooden shield. You are on a quest to defeat the evil dragon of Izir. You've heard he lives up at the north of the kingdom. You set on the path to defeat him and walk into a dark forest.",
#      "You quickly grab your dagger and rush outside to meet your attackers. The first thing you notice is how many there are. They're all dead now though, so it must be a mistake. You head back inside and find that your servant has been killed",
#      "You are Justin, a knight living in the kingdom of Izir. You have a steel longsword and a wooden shield. You are on a quest to defeat the evil dragon of Izir. You've heard he lives up at the north of the kingdom. You do not set on the path to defeat him and do not walk into a dark jail.",
#      "You said \"You go to a forest\"",
#     "You continue to climb up the mountain until you reach a steep hill",
#     "Instead you walk through the gate and into the town of Castle Krakowice",
#     "You run back into the forest",
#     "You return to the forest",
#     "When you arrive at the keep",
#     "You decide to head back to the forest",
    "You turn around and ride back to the mountain range."
]

In [188]:
# Quick experimentation to see if I can find stuff location
LOCATIONS = ["outside", "forest", "jail", "mountain", "town", "village", "keep"]
VERBS = ["rush", "go", "enter", "ride", "arrive", "reach", "walk", "visit", "climb", "run", "return", "head"]
PREPOSITIONS = ["to", "into", "across", "through", "over", "up", "back", "at"]
patterns = [ [{"LEMMA": x}] for x in LOCATIONS]

In [189]:
from spacy.matcher import Matcher
matcher = Matcher(nlp.vocab)

In [190]:
matcher.add("LOCATION", None, *patterns)

In [191]:
def recursive_subject_search(head, subject):
    is_you = False
    subjs = [t for t in head.children if t.dep_ == "nsubj"]
    if len(subjs) == 0:
        while len(subjs) is 0:
            head = head.head
            subjs = [t for t in head.children if t.dep_ == "nsubj"]
            if  head.head == head:
                break
    if subject in [s.text.lower() for s in subjs]:
        return True

In [192]:
def check_negation(head):
    negations = [t for t in head.children if t.dep_ == "neg"]
    return len(negations) > 0

In [195]:
for i, t in enumerate(TEXTS):
    doc = nlp(t)
    matches = matcher(doc)
    locations = []
    for m, start, end in matches:
        span = doc[start:end]
        root = span.root
        print(root)
        print(root.head)
        if root.head.pos_ == "VERB" and root.head.lemma_ in VERBS:
            # Direct child of the verb easy
            head = root.head
            # check for negations
            if not check_negation(head):
                # now need to check if it's "you"
                if recursive_subject_search(head, "you"):
                    print(head.tag_)
                    locations.append(span.root.lemma_)
        elif root.dep_ in ["pobj"]:
            print(root)
            # into, through, acros <Location>
            preposition = root.head
            print(preposition)
            if preposition.text in PREPOSITIONS:
                # it's one of those!
                # get the verb (hopefully)
                head = preposition.head
#                 print(head.dep_)
                if head.pos_ == "VERB" and head.lemma_ in VERBS:
                    if not check_negation(head):
                        if recursive_subject_search(head, "you"):
                            locations.append(span.root.lemma_)
                elif (head.dep_ == "prep" or head.dep_ == "advmod") and head.text in PREPOSITIONS:
                    head = head.head
                    if head.pos_ == "VERB" and head.lemma_ in VERBS:
                        if not check_negation(head):
                            if recursive_subject_search(head, "you"):
                                locations.append(span.root.lemma_)
                
    print(f"Locations for doc {i+1}: ", locations)
            

mountain
range
Locations for doc 1:  []


In [182]:
doc = nlp("You set on the path to defeat him and walk into a dark forest.")
tok = [token for token in doc if token.dep_ == 'nsubj']

In [183]:
t = tok[0]

In [184]:
list(list(t.head.children)[2].children)

[to, him, and, walk]

In [48]:
t.head == t.head.head

True